In [1]:
# imports 
import random
import copy
import re
import os
import sys
import numpy as np
import wandb
from dotenv import load_dotenv

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence

from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

sys.path.append('../')

from utils.dataset import *


In [2]:
dataset = BasicEdgePredDataset('../data/test')

In [3]:
dataset[0]

{'id': 0,
 'question': 'Is there an edge between nodes 5 and 7?',
 'answer': 'no',
 'desc': 'Consider the graph with the following edges: G1 is associated with G2, G1 is associated with G3, G1 is associated with G4, G1 is associated with G5, G2 is associated with G6, G2 is associated with G8, G3 is associated with G4, G3 is associated with G7, G3 is associated with G10, G4 is associated with G7, G4 is associated with G10, G6 is associated with G8, G9 is associated with G10, node_id 0 is G1, node_id 1 is G10, node_id 2 is G2, node_id 3 is G3, node_id 4 is G4, node_id 5 is G5, node_id 6 is G6, node_id 7 is G7, node_id 8 is G8, node_id 9 is G9, layer 0 is from coexpression-heart'}